In [1]:
import sys, os, random
from fractions import Fraction
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx.pauliweb import PauliWeb, compute_pauli_webs
from pyzx import VertexType
zx.settings.colors = zx.rgb_colors

This notebook demonstrates using PyZX's methods for automatically computing and drawing bounded Pauli webs. A Pauli web (a.k.a. _correlation surface_) is a labelling of the edges of ZX-diagram from the set $\{I, X, Y, Z\}$ with certain properties. To explain these properties, it is useful to make some definitions.

To account for Hadmard edges, we label "half-edges" of a diagram. For a pair of connected vertices `v`, `w`, we represent the half-edge closest to `v` as `(v,w)` and the half-edge closest to `w` as `(w,v)`.

We say a spider `v` is _stabilised_ by a Pauli web if `v` (considered as a state) is a +1 eigenstate of the Pauli string labelling its adjacent half-edges and we say `v` is _anti-stabilised_ by a Pauli web if it is a -1 eigenstate of its adjacent half-edges.

A Pauli web is said to be _closed_ if:
1. every spider `v` in the ZX-diagram is stabilised or anti-stabilised
2. for every half-edge `(v,w)` labelled by a Pauli $P$, the other half-edge `(w,v)` is labelled by $P$ for a simple edge and $HPH$ for a Hadamard edge

Otherwise, we say a Pauli web is _open_. The spiders and edges violating conditions 1 and 2 above are called the _boundary_ of a Pauli web.

The function `compute_pauli_webs` automitically associates each non-input vertex `v` in the diagram to an integer `order[v]` giving a time-ordering, and one or two bounded Pauli webs, with the following properties:
1. the boundary of the web consists of only `v` itself, earlier non-Clifford spiders `w` (`order[v] < order[w]>`), and inputs
2. Z spiders have a web `zweb[v]` with a Z-colored edge incident to `v`
3. X spiders have a web `xweb[v]` with a X-colored edge incident to `v`
4. output vertices have two webs `zweb[v]` and `xweb[v]` corresponding to both colors at `v`

Under the hood, this is using PyZX's gflow-finding algorithm to compute a focussed Pauli flow and translate this data into Pauli webs.

# CNOT Examples

In [2]:
# Here's a simple example of a single CNOT gate

c = zx.qasm("""
qreg q[2];
cx q[0], q[1];
""")
g = c.to_graph()
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True)


In [3]:
# There are 4 Pauli webs corresponding to the two outputs
zx.draw(g, labels=True, pauli_web=xwebs[4])
zx.draw(g, labels=True, pauli_web=xwebs[5])
zx.draw(g, labels=True, pauli_web=zwebs[4])
zx.draw(g, labels=True, pauli_web=zwebs[5])

In [4]:
# Pauli webs are computed in the backwards (i.e. Heisenberg-style) by default,
# propegating Paulis from outputs to inputs. Paulis can be propegated forward
# instead by passing backwards=False.

order, zwebsf, xwebsf = compute_pauli_webs(g, backwards=False)
zx.draw(g, labels=True, pauli_web=xwebsf[0])
zx.draw(g, labels=True, pauli_web=xwebsf[1])
zx.draw(g, labels=True, pauli_web=zwebsf[0])
zx.draw(g, labels=True, pauli_web=zwebsf[1])

In [5]:
# Here's an example with more CNOT gates

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
cx q[0], q[2];
cx q[2], q[1];
cx q[0], q[1];
cx q[2], q[0];
""")
g = c.to_graph()

zx.draw(g, labels=True)

In [6]:
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[15])
zx.draw(g, labels=True, pauli_web=zwebs[15])
zx.draw(g, labels=True, pauli_web=xwebs[16])
zx.draw(g, labels=True, pauli_web=zwebs[16])
zx.draw(g, labels=True, pauli_web=xwebs[17])
zx.draw(g, labels=True, pauli_web=zwebs[17])

In [7]:
# Alternatively, we can fuse all the spiders of the same color together to get a
# more interesting diagram.
zx.spider_simp(g)
zx.draw(g, labels=True)

In [8]:
# Since this describes the same unitary, the 6 output webs should all have the same support on
# the inputs as before.
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[15])
zx.draw(g, labels=True, pauli_web=zwebs[15])
zx.draw(g, labels=True, pauli_web=xwebs[16])
zx.draw(g, labels=True, pauli_web=zwebs[16])
zx.draw(g, labels=True, pauli_web=xwebs[17])
zx.draw(g, labels=True, pauli_web=zwebs[17])

# Clifford examples

In [9]:
# Next, we'll look at a single H gate
c = zx.qasm("""
qreg q[1];
h q[0];
""")
g = c.to_graph()

# PyZX renders this as a single H-edge connected to an identity spider
zx.draw(g)

# we don't really need this id-spider, so we can remove it with id_simp
zx.id_simp(g)
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)

In [10]:
# The single output has two Pauli webs
zx.draw(g, labels=True, pauli_web=zwebs[2])
zx.draw(g, labels=True, pauli_web=xwebs[2])

# Note that on a hadamard edge, the color changes in the middle of the edge. To handle this, Pauli webs actually label every
# "half-edge" with a Pauli. (v,w) means the half of the edge touching v, whereas (w,v) means the half of the edge touching w.

#  See for example:
print(zwebs[2].half_edges())

{(0, 2): 'X', (2, 0): 'Z'}


In [11]:
# Here's an example mixing CNOT and H gates

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
h q[1];
h q[2];
cx q[1], q[2];
cx q[0], q[2];
h q[0];
cx q[2], q[1];
h q[1];
h q[1];
cx q[0], q[1];
cx q[2], q[0];
""")
g = c.to_graph()

zx.draw(g, labels=True)

In [12]:
# Note the we get some Y-edges appear, which are shown in a third color
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[20])
zx.draw(g, labels=True, pauli_web=zwebs[20])
zx.draw(g, labels=True, pauli_web=xwebs[21])
zx.draw(g, labels=True, pauli_web=zwebs[21])
zx.draw(g, labels=True, pauli_web=xwebs[22])
zx.draw(g, labels=True, pauli_web=zwebs[22])

In [13]:
# Next, we'll look at a single S gate
c = zx.qasm("""
qreg q[1];
s q[0];
""")
g = c.to_graph()
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[2])

In [14]:
# Since we can handle CNOT, H, and S, we can now compute Pauli webs for arbitrary Clifford
# unitaries.

random.seed(1337)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=4, depth=25, clifford=True)
g = c.to_graph(compress_rows=False)
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, pauli_web=zwebs[41])
zx.draw(g, pauli_web=xwebs[41])

In [15]:
# As before, we can simplify the diagram without changing how Pauli webs behave on the boundaries
g1 = g.copy()

# Calling full_reduce will turn any Clifford circuit into GSLC form, consisting of an input row
# of Clifford Z-spiders connected to an output row of Clifford Z-spiders via Hadamard edges.
# Inputs and outputs can be Hadamard or normal edges.
zx.full_reduce(g1)
g1.normalize()

# nudge some nodes to left, for visibility
for v in [0,1,2,3,11]: g1.set_row(v, g1.row(v)-1)
zx.draw(g1, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g1)

zx.draw(g1, pauli_web=zwebs[41])
zx.draw(g1, pauli_web=xwebs[41])

# Errors and corrections

Before moving on to Clifford+T examples, we'll show how to introduce errors into ZX-diagrams, correct them, and check the corrected diagram is equal to the original, error-free diagram, up to a global phase.

We can use the `PauliWeb` class to represent arbitrary Pauli errors on half edges. Note this will _not_ usually be a valid, closed Pauli web, but we can still use the `PauliWeb` class to hold this data. At some point, it might be worth adding an `is_valid` method to `PauliWeb` to check the Pauli web conditions are satisfied.

The kinds of errors we are interested in here are logical flips coming from lattice surgery operations. Normally, these will non-deterministically introduce X errors next to Z nodes and Z errors next to X nodes.

Since we know at runtime where these errors are, they are always correctable. Hence, the error rate and which subset of errors _actually_ occur is irrelevant. To demonstrate the corrections for arbitrary errors, we will call `PauliWeb.random` with its default arguments, which introduces a uniformly random Pauli X, Y, or Z with 30% probability on every half-edge.

In [16]:
random.seed(1337)
g = zx.generate.CNOT_HAD_PHASE_circuit(qubits=4, depth=10, clifford=True).to_graph(compress_rows=False)
zx.draw(g, labels=True)

# introduce random Pauli errors on edges and draw them
errors = PauliWeb.random(g)
zx.draw(g, pauli_web=errors)

In [17]:
# The `PauliWeb.graph_with_errors()` method returns a copy of the original graph, where
# Pauli errors are inserted as pi-spiders on the edges colored by the web.
zx.draw(errors.graph_with_errors())

In [18]:
# To find the appropriate corrections, we should introduce Pauli X and Z gates at the outputs
# to make all 8 output webs commute with `errors`.

order, zwebs, xwebs = compute_pauli_webs(g)
output_es = [(o, next(iter(g.neighbors(o)))) for o in g.outputs()]


errors1 = errors.copy()

# for each of the outputs, if the errors anti-commute with the associated Z-web, introduce
# an X-error at that output to make it commute, and do similar for X-webs.
for o,n in output_es:
    print(f'Z-web for output {o}: ', end='')
    if zwebs[o].commutes_with(errors1):
        print('commutes')
    else:
        print('anti-commutes', end='')
        errors1.add_half_edge((o,n), 'X')
        if zwebs[o].commutes_with(errors1):
            print(', corrected')
    print(f'X-web for output {o}: ', end='')
    if xwebs[o].commutes_with(errors1):
        print('commutes')
    else:
        print('anti-commutes', end='')
        errors1.add_half_edge((o,n), 'Z')
        if xwebs[o].commutes_with(errors1):
            print(', corrected')

Z-web for output 20: commutes
X-web for output 20: anti-commutes, corrected
Z-web for output 21: anti-commutes, corrected
X-web for output 21: anti-commutes, corrected
Z-web for output 22: commutes
X-web for output 22: commutes
Z-web for output 23: anti-commutes, corrected
X-web for output 23: anti-commutes, corrected


In [19]:
# All the errors have now been corrected, so the graph with Pauli errors given by `errors1` should
# be equal, up to a global phase to the origin graph
ge = errors1.graph_with_errors()
zx.draw(ge)

# ...so if we compose `ge` with the adjoint of `g`, we should get the identity
h = g.adjoint() * ge
zx.full_reduce(h)
h.pack_circuit_rows()
zx.draw(h)

# Clifford+T Examples

Now, we'll move to some Clifford+T examples.

We will handle non-Clifford spiders by introducing additional outputs that have been measured in the basis $\{|T\rangle, Z|T\rangle \}$, where $|T\rangle := T|{+}\rangle$ is a $T$ magic state. If the spider has degree d > 1, we unfuse a 1-legged spider, which we treat as the "T-basis" measurement, connected to an d+1 legged spider with no phase.

In [20]:
g = zx.Graph()
g.add_vertex(qubit=0, row=0)
g.add_vertex(qubit=1, row=0)
g.add_vertex(qubit=2, row=0)
g.add_vertex(qubit=0, row=2)
g.add_vertex(qubit=2, row=2)
g.add_vertex(VertexType.Z, qubit=1, row=1, phase=Fraction(1/4))
for b in range(5): g.add_edge((5,b))

# a 5-legged pi/4 spider
zx.draw(g)

# a 6-legged spider with no phase, connected to a T-basis measurement
zx.simplify.unfuse_non_cliffords(g)
zx.draw(g)

There are many equivalent ways to implement a $\{|T\rangle, Z|T\rangle \}$ measurement fault-tolerantly in lattice surgery. One way to do this is to first perform a Z-merge between a data qubit and a $|T\rangle$ magic state, then depending on the outcome of the merge, measure the remaining qubit in the X or Y basis.

In [21]:
# Merge with parity 0, followed by an X measurement
g = zx.Graph()
g.add_vertex(qubit=0, row=0)
g.add_vertex(VertexType.Z, qubit=2, row=0, phase=Fraction(1/4))
g.add_vertex(VertexType.Z, qubit=1, row=1)
g.add_vertex(VertexType.Z, qubit=1, row=2)
for e in [(0,2), (1,2), (2,3)]: g.add_edge(e)
zx.draw(g, labels=True)

# ...is equivalent to a |T> measurement
zx.full_reduce(g)
zx.draw(g)

In [22]:
# Merge with parity 1, followed by an Y measurement
h = zx.Graph()
h.add_vertex(qubit=0, row=0)
h.add_vertex(VertexType.Z, qubit=2, row=0, phase=Fraction(1/4))
h.add_vertex(VertexType.Z, qubit=1, row=1)
h.add_vertex(VertexType.Z, qubit=1, row=2, phase=Fraction(1/2))
h.add_vertex(VertexType.X, qubit=1.5, row=0.5, phase=1)
for e in [(0,2), (1,4), (4,2), (2,3)]: h.add_edge(e)
zx.draw(h, labels=True)

# ...is also equivalent to a |T> measurement
zx.full_reduce(h)
zx.draw(h)

In both cases above, we show the +1 outcome of the X/Y measurement. Getting the other outcome yields the same effect, up to a Pauli Z error, which can be corrected later.

Using this primitive, we now look at implementing a single T gate.

In [23]:
c = zx.qasm("""
qreg q[1];
t q[0];
""")
g = c.to_graph()
zx.draw(g)

# Unfuse the non-Clifford spiders, so they look like outputs measured in {|T>, Z|T>}
zx.simplify.unfuse_non_cliffords(g)
zx.draw(g, labels=True)

In [24]:
# The relevant Pauli webs for this example are the Z and X webs of the output 2
# as well as the Z-web of the non-Clifford spider 3.

order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=zwebs[2])
zx.draw(g, labels=True, pauli_web=xwebs[2])
zx.draw(g, labels=True, pauli_web=zwebs[3])

In [25]:
# As we did in the Clifford examples, we can introduce random Pauli errors on all the edges
random.seed(1330)
errors = PauliWeb.random(g)
zx.draw(g, pauli_web=errors)

errors1 = errors.copy()

# To implement the T-measurement at node 3, we need to look at the associated Z-web of 3
# If it anti-commutes with the errors, we should apply an X correction before we measure, or equivalently,
# flip whether we are measuring X or Y in the fault-tolerant gadget that injects a magic state.
if not zwebs[3].commutes_with(errors):
    errors1.add_half_edge((3,1), 'X')

# a single Y error indeed anti-commutes with zweb[3], so a correction appears
zx.draw(g, pauli_web=errors1)



In [26]:
errors2 = errors1.copy()

# The T-measurement itself will either apply <T| or <T|Z with uniform probability. We can simulate this by
# randomly introducing a Z error next to node 3.
if random.random() > 0.5:
    errors2.add_half_edge((v, n), 'Z')

# Finally, we apply corrections on the output just like we did in the Clifford case
if not zwebs[2].commutes_with(errors2): errors2.add_half_edge((2,1), 'X')
if not xwebs[2].commutes_with(errors2): errors2.add_half_edge((2,1), 'Z')

zx.draw(g, pauli_web=errors2)

# The result, after we have applied corrections, should indeed be a single T gate
ge = errors2.graph_with_errors()
zx.full_reduce(ge)
zx.draw(ge)

The next example is a circuit with some CNOT gates and a T gate, which can be simplified to a single phase gadget. I'm doing this manually here, since the automated simplifier comes up with a different answer (which is equivalent to this one, up to local Cliffords, but less clear what is going on).

In [27]:
c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
t q[2];
cx q[1], q[2];
cx q[0], q[1];
""")
zx.draw(c)

# manual ZX simplification to get a single phase gadget
g = c.to_graph()
zx.simplify.unfuse_non_cliffords(g)
zx.basicrules.strong_comp(g, 5, 7)
zx.simplify.spider_simp(g, quiet=True)
zx.basicrules.strong_comp(g, 3, 6)
zx.simplify.spider_simp(g, quiet=True)
zx.simplify.id_simp(g, quiet=True)

zx.draw(g, labels=True)

d = dict()

# pauli web calculation
order, zwebs, xwebs = compute_pauli_webs(g, debug=d)

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=zwebs[15])

This and the output Pauli webs is enough data to simulate running this computation and computing corrections. Rather than doing that explicitly, lets go straight to a random Clifford+T circuit.

In [28]:
# Generate a random CNOT, H, T circuit
random.seed(1330)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=3, depth=30)
g = c.to_graph(compress_rows=False)
zx.draw(g)

# Unfuse the non-Clifford spiders, so they look like outputs measured in {|T>, Z|T>}
zx.simplify.unfuse_non_cliffords(g)
zx.draw(g, labels=True)

In [29]:
# as before, we can introduce random Pauli errors into the graph
random.seed(1337)
errors = PauliWeb.random(g)
zx.draw(g, pauli_web=errors)

In [30]:
# however, the correction strategy is more elaborate, since we have internal non-Clifford
# gadgets to handle. First, compute Pauli webs:

order, zwebs, xwebs = compute_pauli_webs(g)

# `order` tells us the order in which we should measure non-Clifford spiders. For Clifford diagrams,
# it was empty, since there were no non-Clifford spiders. In this example, it shows that we need to
# split the non-Clifford spiders into 2 time steps, where the last 4 measurements might depend on the
# outcomes of the first 4.
print(order)

{51: 1, 52: 1, 53: 1, 54: 1, 55: 2, 56: 2, 57: 2, 58: 2}


In [31]:
# For example, if we look at the Z-web of spider 55, it touches 51 and 52, which means we need
# to be done with those spiders first.
zx.draw(g, labels=True, pauli_web=zwebs[55])

In [32]:
errors1 = errors.copy()

# traverse through the T spiders in the order given by `order`
for v in sorted(order.keys(), key=lambda v: order[v]):
    n = next(iter(g.neighbors(v)))

    # if the Z-web associated with that spider anti-commutes with previous errors,
    # introduce an X error before the measurement. This has the same effect, up to Paulis
    # as choosing to measure Y rather than X in the fault-tolerant gadget
    if not zwebs[v].commutes_with(errors1):
        errors1.add_half_edge((v, n), 'X')
    
    # "do" the measurement, i.e. plug in |T> or Z|T> with uniform probability
    if random.random() > 0.5:
        errors1.add_half_edge((v, n), 'Z')

# finally, use the output webs to correct outputs, as before
for o in g.outputs():
    n = next(iter(g.neighbors(o)))
    if not zwebs[o].commutes_with(errors1):
        errors1.add_half_edge((o,n), 'X')
    if not xwebs[o].commutes_with(errors1):
        errors1.add_half_edge((o,n), 'Z')

# here is the original web
zx.draw(g, pauli_web=errors)

# here is the corrected web
zx.draw(g, pauli_web=errors1)

# n.b. we can multiply these two to just show the corrections
zx.draw(g, pauli_web=errors*errors1)

In [33]:
h = g.adjoint() * errors1.graph_with_errors()
zx.full_reduce(h)
h.pack_circuit_rows()
zx.draw(h)

In [34]:
# The same procedure works with a reduced ZX-diagram, rather than a circuit, as input
random.seed(1337)
g = zx.generate.CNOT_HAD_PHASE_circuit(qubits=3, depth=40).to_graph()
zx.draw(g)
zx.full_reduce(g)
g.normalize()

# for simplicity, we'll make sure all the non-Clifford spiders are Z
zx.to_rg(g, init_z=set(v for v in g.vertices() if not zx.utils.phase_is_clifford(g.phase(v))))

zx.simplify.unfuse_non_cliffords(g)
zx.draw(g, labels=True)

In [35]:
random.seed(1337)
order, zwebs, xwebs = compute_pauli_webs(g)
errors = PauliWeb.random(g)
errors1 = errors.copy()

# "simulate" the diagram and compute corrections, as before
for v in sorted(order.keys(), key=lambda v: order[v]):
    n = next(iter(g.neighbors(v)))
    if not zwebs[v].commutes_with(errors1): errors1.add_half_edge((v, n), 'X')
    if random.random() > 0.5: errors1.add_half_edge((v, n), 'Z')
for o in g.outputs():
    n = next(iter(g.neighbors(o)))
    if not zwebs[o].commutes_with(errors1): errors1.add_half_edge((o,n), 'X')
    if not xwebs[o].commutes_with(errors1): errors1.add_half_edge((o,n), 'Z')

zx.draw(g, pauli_web=errors)
zx.draw(g, pauli_web=errors*errors1)

h = g.adjoint() * errors1.graph_with_errors()
zx.full_reduce(h)
h.pack_circuit_rows()
zx.draw(h)